In [17]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from sklearn.preprocessing import OneHotEncoder 
from tqdm import tqdm
from keras import layers
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os
import random

In [18]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [19]:
BASE_PATH = '/content/gdrive/MyDrive/Colab Notebooks/Brain Tumor Classification/'

AUGMENTED_TRAIN_PATH = BASE_PATH + 'augmented_dataset/train/'

AUGMENTED_VALIDATION_PATH = BASE_PATH + 'augmented_dataset/validation/'

AUGMENTED_TEST_PATH = BASE_PATH + 'augmented_dataset/test/'

TARGET_IMAGE_SIZE = (224, 224)

In [20]:
def read_dataset(path):
    dataset = []
    for feature_class in tqdm(os.listdir(path)):
      if not feature_class.startswith("."):
          feature_class_path = path + feature_class
          for (i, file_name) in enumerate(os.listdir(feature_class_path)):
              file_path = feature_class_path + '/' + file_name
              image = cv2.imread(file_path)
              image_and_class = (image, feature_class)
              dataset.append(image_and_class)
    return dataset

In [21]:
training_dataset = read_dataset(AUGMENTED_TRAIN_PATH)
random.shuffle(training_dataset)

100%|██████████| 2/2 [00:05<00:00,  2.95s/it]


In [22]:
validation_dataset = read_dataset(AUGMENTED_VALIDATION_PATH)
random.shuffle(validation_dataset)

100%|██████████| 2/2 [00:01<00:00,  1.29it/s]


In [23]:
testing_dataset = read_dataset(AUGMENTED_TEST_PATH)
random.shuffle(testing_dataset)

100%|██████████| 2/2 [00:01<00:00,  1.17it/s]


In [24]:
def count_class_labels(dataset, dataset_type):
    yes_count = 0
    no_count = 0
    for data in dataset:
        label = data[1]
        if label == 'yes':
            yes_count = yes_count + 1
        else:
            no_count = no_count + 1
    print("Number of YES labels in the {0} dataset are {1}".format(dataset_type, yes_count))
    print("Number of NO labels in the {0} dataset are {1}".format(dataset_type, no_count))
    print("---")

In [25]:
count_class_labels(training_dataset, "training")
count_class_labels(validation_dataset, "validation")
count_class_labels(testing_dataset, "testing")

Number of YES labels in the training dataset are 519
Number of NO labels in the training dataset are 345
---
Number of YES labels in the validation dataset are 124
Number of NO labels in the validation dataset are 60
---
Number of YES labels in the testing dataset are 104
Number of NO labels in the testing dataset are 79
---


In [26]:
encoder = OneHotEncoder()
encoder.fit([[0], [1]]) 

OneHotEncoder()

In [27]:
def split_into_X_and_y(dataset):
    X = []
    y = []
    for data in dataset:
        X.append(data[0])
        label = data[1]
        if label == 'yes':
            y.append(encoder.transform([[1]]).toarray())
        elif label == 'no':
            y.append(encoder.transform([[0]]).toarray())

    return np.array(X), np.array(y).reshape(len(y), 2)

In [28]:
X_train, y_train = split_into_X_and_y(training_dataset)
X_validation, y_validation = split_into_X_and_y(validation_dataset)
X_test, y_test = split_into_X_and_y(testing_dataset)

In [39]:
KERNEL_SIZE = (2, 2)
STRIDES = (2, 2)
ACTIVATION_FUNCTION_RELU = 'relu'
PADDING_SAME = 'same'
DROPOUT = 0.25

model = Sequential()

# Block 1
model.add(Conv2D(32, kernel_size = KERNEL_SIZE, padding = PADDING_SAME, input_shape = (224, 224, 3)))
model.add(Conv2D(32, kernel_size = KERNEL_SIZE,  activation = ACTIVATION_FUNCTION_RELU, padding = PADDING_SAME))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(DROPOUT))

# Block 2
model.add(Conv2D(64, kernel_size = KERNEL_SIZE, activation = ACTIVATION_FUNCTION_RELU, padding = PADDING_SAME))
model.add(Conv2D(64, kernel_size = KERNEL_SIZE, activation = ACTIVATION_FUNCTION_RELU, padding = PADDING_SAME))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = KERNEL_SIZE, strides = STRIDES))
model.add(Dropout(DROPOUT))

# Block 3
model.add(Conv2D(128, kernel_size = KERNEL_SIZE, activation = ACTIVATION_FUNCTION_RELU, padding = PADDING_SAME))
model.add(Conv2D(128, kernel_size = KERNEL_SIZE, activation = ACTIVATION_FUNCTION_RELU, padding = PADDING_SAME))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = KERNEL_SIZE, strides = STRIDES))
model.add(Dropout(DROPOUT))

model.add(Flatten())

model.add(Dense(512, activation = ACTIVATION_FUNCTION_RELU))
model.add(Dropout(0.5))
model.add(Dense(2, activation = 'softmax'))

model.compile(loss = "categorical_crossentropy", optimizer = 'adam', metrics = ['accuracy'])

In [40]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 224, 224, 32)      416       
                                                                 
 conv2d_25 (Conv2D)          (None, 224, 224, 32)      4128      
                                                                 
 batch_normalization_12 (Bat  (None, 224, 224, 32)     128       
 chNormalization)                                                
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 112, 112, 32)     0         
 g2D)                                                            
                                                                 
 dropout_16 (Dropout)        (None, 112, 112, 32)      0         
                                                                 
 conv2d_26 (Conv2D)          (None, 112, 112, 64)     

In [41]:
result = model.fit(X_train, y_train, epochs = 5, batch_size = 50, verbose = 1, validation_data = (X_validation, y_validation))

Epoch 1/5
18/18 [==============================] - 240s 13s/step - loss: 24.1879 - accuracy: 0.6377 - val_loss: 10.7073 - val_accuracy: 0.6793
Epoch 2/5
18/18 [==============================] - 238s 13s/step - loss: 5.3883 - accuracy: 0.6539 - val_loss: 4.6722 - val_accuracy: 0.7446
Epoch 3/5
18/18 [==============================] - 238s 13s/step - loss: 2.2838 - accuracy: 0.7118 - val_loss: 5.3780 - val_accuracy: 0.7011
Epoch 4/5
18/18 [==============================] - 237s 13s/step - loss: 0.8764 - accuracy: 0.7465 - val_loss: 3.8675 - val_accuracy: 0.6793
Epoch 5/5
18/18 [==============================] - 238s 13s/step - loss: 0.5018 - accuracy: 0.7616 - val_loss: 2.9331 - val_accuracy: 0.6848


In [42]:
loss, accuracy = model.evaluate(X_test, y_test)

6/6 [==============================] - 12s 2s/step - loss: 1.3595 - accuracy: 0.7814


In [43]:
accuracy

0.7814207673072815